# 최종코드

In [23]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager 
import matplotlib.font_manager as fm

import datetime as datetime1
import datetime
from datetime import datetime
from collections import Counter

# 워드 파일 레포트로 만들기 위해 필요한 라이브러리 
import docx
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import RGBColor, Pt
from docx.shared import Inches
from io import BytesIO

import os
from os import listdir
from os.path import isfile, join
import glob

In [24]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
print(f)

# matplotlib 폰트 위치
print(mpl.matplotlib_fname())

# 캐시 파일 저장된 폴더 위치
print(mpl.get_cachedir())

['DejaVu Serif Display', 'cmex10', 'STIXGeneral', 'STIXSizeOneSym', 'STIXSizeFiveSym', 'DejaVu Sans', 'DejaVu Sans Mono', 'cmss10', 'DejaVu Sans Display', 'STIXSizeTwoSym', 'STIXSizeTwoSym', 'DejaVu Sans Mono', 'cmb10', 'DejaVu Sans Mono', 'STIXNonUnicode', 'cmsy10', 'STIXNonUnicode', 'DejaVu Sans', 'STIXSizeOneSym', 'STIXSizeThreeSym', 'STIXGeneral', 'STIXSizeFourSym', 'cmmi10', 'STIXGeneral', 'STIXSizeFourSym', 'DejaVu Sans', 'DejaVu Sans', 'DejaVu Serif', 'cmr10', 'STIXGeneral', 'STIXSizeThreeSym', 'STIXNonUnicode', 'cmtt10', 'DejaVu Serif', 'STIXNonUnicode', 'DejaVu Sans Mono', 'DejaVu Serif', 'DejaVu Serif', 'Courier New', 'Trebuchet MS', 'Verdana', 'Gadugi', 'Nirmala UI', 'Sitka Small', 'Microsoft Himalaya', 'Corbel', 'Microsoft Tai Le', 'Times New Roman', 'Microsoft JhengHei', 'Arial', 'Segoe UI Emoji', 'Segoe MDL2 Assets', 'Tahoma', 'Franklin Gothic Medium', 'Trebuchet MS', 'Sitka Small', 'Palatino Linotype', 'Franklin Gothic Medium', 'Microsoft YaHei', 'Comic Sans MS', 'Palati

In [3]:
# 파일을 열게되면 './2023_06/Report/~$6월 레포트 작성 필요 명단.xlsx' 이런 식으로 파일이 생성되는데 이 파일을 읽어서 오류 발생함

####### 매달 받는 데이터에 따라서 date만 수정해주면 됨 ######
date = '레포트데이터_2024_06/2024_06'

report_file = glob.glob(f'./{date}/Report/*.xlsx') # 레포트 작성 필요명단  
tmds_nts = glob.glob(f'./{date}/TDMS/분당_TMDS_NonTimeSeries_*.xlsx') # NonTimeSeries 
tmds_ts = glob.glob(f'./{date}/TDMS/분당_TMDS_TimeSeries_*.xlsx') # TimeSeries

print(report_file)
print(tmds_nts)
print(tmds_ts)

vital_path = f'./{date}/Vital'

['./레포트데이터_2024_06/2024_06/Report\\2024년6월 레포트 작성 필요 명단.xlsx']
['./레포트데이터_2024_06/2024_06/TDMS\\분당_TMDS_NonTimeSeries_20240712.xlsx']
['./레포트데이터_2024_06/2024_06/TDMS\\분당_TMDS_TimeSeries_20240712.xlsx']


## 레포트 작성을 위한 데이터 준비


### 1. TimeSeries 파일에서 sessionID-PID 매칭코드

In [4]:
#nontime = pd.read_excel(tmds_nts[0])
nontime = pd.read_excel(tmds_nts[0])
unnamed_cols = [col for col in nontime.columns if col.startswith('Unnamed:')]
nontime = nontime.drop(columns=unnamed_cols)

# 24/1월 P_ID와 P_NAME이 바뀌어서 옴.. 이번에만 추가함
#nontime.rename(columns={'P_ID':'P_Name', 'P_Name':'P_ID'}, inplace=True)

time = pd.read_excel(tmds_ts[0])
# sessionID가 NaN인 것 제거 (23.12.11 추가됨)
time.dropna(subset=['SessionID'], inplace=True)

# 23.12월용 임시
# time = time[time['SessionID'] != 21223]

nontime_list = list(set(nontime['SessionID']))
time_list = list(set(time['SessionID']))

nontime_dict = nontime.to_dict()
session_id_pid_dict = {}
for idx in nontime_dict['SessionID'].keys():
  session_id_pid_dict[nontime_dict['SessionID'][idx]] = nontime_dict['P_ID'][idx]
  
time_id_dict = []
for s_id in time.SessionID: # 12월 데이터에는 sessionID가 NaN인게 존재 
  time_id_dict.append(session_id_pid_dict[s_id])
time['P_ID']=time_id_dict

# sessionID에 맞는 PID 컬럼 추가 후 새로운 파일로 저장 

time.to_excel(tmds_ts[0])
print("SessionID & PID matching finished !")

SessionID & PID matching finished !


### 2. 많은 column들 중 사용하는 column만 남기기 

In [5]:
# 파일을 dataframe으로 
report_df = pd.read_excel(report_file[0], usecols='A', names=['P_ID']) 
nts_df = pd.read_excel(tmds_nts[0])

# 23.12용 추가된 코드 
#unnamed_cols = [col for col in nts_df.columns if col.startswith('Unnamed:')]
#nts_df = nts_df.drop(columns=unnamed_cols)

# 24/1월 P_ID와 P_NAME이 바뀌어서 옴.. 이번에만 추가함
#nts_df.rename(columns={'P_ID':'P_Name', 'P_Name':'P_ID'}, inplace=True)
########

# time series에서는 sessionID, P_ID, STime, VVitalParameter_Systolic, VVitalParameter_Diastolic, VVitalParameter_Pulse 사용
ts_parameter = ['SessionID', 'P_ID', 'STime', 'VVitalParameter_Systolic', 'VVitalParameter_Diastolic', 'VVitalParameter_Pulse', 'UF_Volume']
ts_df = pd.read_excel(tmds_ts[0])
ts_df = ts_df[ts_parameter]

# P_ID가 일치하는 row만 살리기 
# sessionID는 시간 순으로 되어있음 
# total_df = 필요한것만 남겨둔 dataframe (time_series + non_time_series)
total_df = pd.merge(ts_df, report_df, on='P_ID')
total_df = pd.merge(total_df, nts_df, on=['P_ID', 'SessionID'])

#### 레포트 작성해야하는 환자에 대한 time_series, non time_series 데이터 있는지 확인

In [6]:
### check report_pid and nts, ts patient id
report_pid_list = list(report_df['P_ID'])
print("레포트를 작성해야하는 환자 리스트: ", len(report_pid_list))

nts_pid_list = list(set(nts_df['P_ID']))
print("nontime series pid :", nts_pid_list, "\n총: ", len(nts_pid_list))

ts_pid_list = list(set(ts_df['P_ID']))
print("time series pid :", ts_pid_list, "\n총: ", len(ts_pid_list))

# ts_pid_list와 nts_pid_list가 일치하는지 확인 , TRUE
print("ts pid list와 nts pid list가 일치 여부: ", np.array_equal(nts_pid_list, ts_pid_list))

print("nts_pid_list에만 있는 것", list(set(nts_pid_list) - set(report_pid_list)))
print("report_pid_list에만 있는 것", list(set(report_pid_list) - set(nts_pid_list)))
print("두 list의 교집합: ", len(list(set(report_pid_list) & set(nts_pid_list))), list(set(report_pid_list) & set(nts_pid_list)))

레포트를 작성해야하는 환자 리스트:  42
nontime series pid : [14648836, 16248838, 21792647, 28277512, 36946314, 28134538, 28292751, 12792977, 11208695, 29015700, 10680597, 22064408, 25143836, 19054621, 23777567, 33014946, 22674470, 12445992, 12883114, 11545642, 16765869, 11657901, 30329904, 10540978, 12281011, 11345974, 17552439, 16743737, 14566972, 26695741, 13172413, 11593021, 17813444, 32086599, 12344145, 20417873, 28190932, 36163544, 11268060, 38335585, 22939362, 23888227, 15475435, 20280305, 25257588, 23922165, 18470516, 18426999, 12558069, 12879867, 16586749, 23616895] 
총:  52
time series pid : [14648836, 16248838, 21792647, 28277512, 36946314, 28134538, 28292751, 12792977, 11208695, 29015700, 10680597, 22064408, 25143836, 19054621, 23777567, 33014946, 22674470, 12445992, 12883114, 11545642, 16765869, 11657901, 30329904, 10540978, 12281011, 11345974, 17552439, 16743737, 14566972, 26695741, 13172413, 11593021, 17813444, 32086599, 12344145, 20417873, 28190932, 36163544, 11268060, 38335585, 2293936

In [ ]:
# 이전에 P_Name이 없이 와서 추가한 코드 

# total_df.drop(columns=['P_Name'], inplace=True)
# print(total_df.columns)
# report_df_for_name = pd.read_excel(report_file[0]).rename(columns={'P_NAME':'P_Name'})
# report_df_for_name = report_df_for_name[['P_ID', 'P_Name']]

# total_df = pd.merge(total_df, report_df_for_name, on='P_ID')
# len(total_df.columns)

### 3. vital 파일 (산소포화도) 

#### 산소포화도 처리 관련 함수

In [7]:
### 입력시간에 대해서 일, 시:분:초 로 나타내는 함수 

def make_time_str(td):
    days = td.days
    hours, remainder = divmod(td.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)    
    if len(str(hours))==1:
        hours = '0'+str(hours)
    if len(str(minutes))==1:
        minutes = '0'+str(minutes)
    if len(str(seconds))==1:
        seconds = '0'+str(seconds)       
    if days == 0:
        time_str =f'{hours}:{minutes}:{seconds}'
    else:    
        time_str = f'{days}일, {hours}:{minutes}:{seconds}'
    return time_str

# 한 환자의 session date 뽑아오는 함수 
def get_session_dates(pid):
    time_df = total_df[total_df['P_ID'] == pid].dropna(subset=['VVitalParameter_Systolic', 'VVitalParameter_Diastolic', 'VVitalParameter_Pulse'])['STime'] # 해당 pid의 date 
    # 원래 s_time이 2024.3.1  7:26:47 AM의 형식인데 여기서 연, 월, 일만 가져오는 코드 
    tmp_df = time_df.apply(lambda x: x.strftime("%Y%m%d")[2:])
    time_list = list(set(tmp_df.values.tolist()))
    return time_list

In [8]:
set(total_df[total_df['P_ID'] == 11657901]['SessionID'])

{25649, 25699, 25801, 25851, 25953, 26002, 26103, 26155, 26252}

#### 환자별 vital 파일을 모두 읽고 산소포화도 95% 미만인 비율 구하는 부분


In [9]:
pid_file = os.listdir(vital_path) # pid + 환자이름

if '.DS_Store' in pid_file: # 가끔 압축풀고나서 이런 파일 있으면 오류남
    pid_file.remove('.DS_Store')

pid_spo2_dict = {} # key : pid+환자이름 , value : dict [ key :날짜, value : 날짜별 총 투석 시간, Spo2 저하 시간, 비율]  

for pid_file_ in pid_file: # 한 환자씩 본다 
#for p_idx in range(39, 40):
    #pid_file_ = pid_file[p_idx]
    pid_file_ = pid_file_[2:] if '_' in pid_file_ else pid_file_  # 24.07.10 '._{pid}' 인 경우를 수정
    only_pid = int(pid_file_[:8])  # pid_file_ format이 string임
    #print(only_pid)
    pid_spo2_dict[only_pid] = {}
    pid_path = os.path.join(vital_path, pid_file_)
    
    session_path = os.listdir(pid_path) # 한 환자의 여러 세션
    if '.DS_Store' in session_path: # 가끔 압축풀고나서 이런 파일 있으면 오류남
        session_path.remove('.DS_Store')
    days = [] # 한 사람의 세션 날짜 다 들어있는 리스트
    for session_data in session_path:
        days.append(session_data.split('_')[1])
    days = list(set(days))

    days_from_df = get_session_dates(only_pid) 
    #print("days", days)
    # print("dfd", days_from_df)

    #######  days 매칭 조건 넣기  ######
    final_days = list(set(days) & set(days_from_df)) # intersection으로 겹치는 부분만
    #print("final_days", final_days)

    date_file_dict = {} # key : date, value : 그에  상응하는 file 이름
    for date in final_days: 
        date_file_dict[date]=[]
    for date in final_days:            
        for session_data in session_path:
            if session_data.split('_')[1] == date:
                date_file_dict[date].append(session_data)
    
    # 환자의 한달 동안의 총 시간                 
    whole_total_spo2_under_95_sec = datetime1.timedelta(0)
    whole_total_time = datetime1.timedelta(0)                
    
    
    for date in date_file_dict.keys(): # 각 날짜의 데이터를 본다 
        if len(date_file_dict[date]) > 1: # 한 session의 파일 두 개 이상이면
            first_pd = pd.read_csv(os.path.join(pid_path, date_file_dict[date][0]))
            for file_ in date_file_dict[date][1:]:
                tmp_pd = pd.read_csv(os.path.join(pid_path, file_))
                first_pd = pd.concat([first_pd,tmp_pd],ignore_index=True)
            first_pd = first_pd.sort_values(by=['Time'])   
            first_pd = first_pd.reset_index() 
        else: # 한 session의 파일 하나면
            first_pd = pd.read_csv(os.path.join(pid_path,date_file_dict[date][0]))     
    
        # first_pd : 한 환자의 하루의 데이터                         
        data_dict = first_pd.to_dict()
        
        for time_idx,idx in enumerate(data_dict['Time'].keys()):
            # 맨 앞 두 point의 시간 간격 구해봄.
            if time_idx ==0:
                curr_time = datetime.strptime(data_dict['Time'][idx],'%Y-%m-%d %H:%M:%S.%f')
                next_time = datetime.strptime(data_dict['Time'][time_idx+1],'%Y-%m-%d %H:%M:%S.%f')  
                delta_time = next_time-curr_time

        under95_data = first_pd[first_pd.PLETH_SPO2<95] # 한 세션 중 95 이하인 시간    
        total_spo2_under_95_sec = len(first_pd[first_pd.PLETH_SPO2<95])*delta_time # 한 세션 중 95 이하인 시간  -> second로 

        whole_total_spo2_under_95_sec += total_spo2_under_95_sec # 한달 데이터에도 적립

        #start_time = datetime.strptime(data_dict['Time'][0],'%Y-%m-%d %H:%M:%S.%f')
        #end_time = datetime.strptime(data_dict['Time'][len(data_dict['Time'])-1],'%Y-%m-%d %H:%M:%S.%f')  
        #total_time = end_time-start_time # 한 세션 동안의 총 투석시간 
        total_time = len(first_pd)*delta_time
        
        under95_percent = round((total_spo2_under_95_sec.total_seconds() / total_time.total_seconds())*100, 3)
        
        
        # total_time : 한 세션당 데이터를 측정한 전체 시간 
        # spo2_under_95_sec : 산소포화도가 95% 미만인 시간 
        # spo2_under_95_percent: 전체 시간 대비 산소포화도가 95% 미만인 시간의 비율 
        pid_spo2_dict[only_pid][date] = {'total_time':make_time_str(total_time), 'spo2_under_95_sec':make_time_str(total_spo2_under_95_sec), 'spo2_under_95_percent':under95_percent}
        whole_total_time += total_time


    pid_spo2_dict[only_pid]['whole_total_time'] = make_time_str(whole_total_time)     
    pid_spo2_dict[only_pid]['whole_spo2_under_95_sec'] = make_time_str(whole_total_spo2_under_95_sec)  


    try:    
        pid_spo2_dict[only_pid]['whole_spo2_under_95_percent'] = round((whole_total_spo2_under_95_sec.total_seconds() / whole_total_time.total_seconds())*100, 3) 
        #print(pid_spo2_dict[pid_file_])
    except ZeroDivisionError:
        pass                   

In [10]:
pid_spo2_dict

{10540978: {'240604': {'total_time': '03:44:23',
   'spo2_under_95_sec': '00:09:59',
   'spo2_under_95_percent': 4.449},
  '240611': {'total_time': '03:55:00',
   'spo2_under_95_sec': '00:04:37',
   'spo2_under_95_percent': 1.965},
  '240618': {'total_time': '03:50:22',
   'spo2_under_95_sec': '00:01:15',
   'spo2_under_95_percent': 0.543},
  'whole_total_time': '11:29:45',
  'whole_spo2_under_95_sec': '00:15:51',
  'whole_spo2_under_95_percent': 2.298},
 10680597: {'240603': {'total_time': '03:56:13',
   'spo2_under_95_sec': '00:00:00',
   'spo2_under_95_percent': 0.0},
  '240628': {'total_time': '03:52:49',
   'spo2_under_95_sec': '00:00:28',
   'spo2_under_95_percent': 0.2},
  '240605': {'total_time': '03:54:26',
   'spo2_under_95_sec': '00:00:08',
   'spo2_under_95_percent': 0.057},
  '240626': {'total_time': '03:51:49',
   'spo2_under_95_sec': '00:00:37',
   'spo2_under_95_percent': 0.266},
  '240614': {'total_time': '03:46:27',
   'spo2_under_95_sec': '00:00:40',
   'spo2_under_9

### 레포트 제작 관련 함수들

In [11]:
### 환자의 session 날짜들을 기준으로 주 별로 나누기 
### 예) 7월 1일 ~ 7월 6일에 해당하는 session 날짜들은 1주차 
### 예) 7월 8일 ~ 7월 12일에 해당하는 session 날짜들은 2주차 

# input: 한 환자의 sessionID 모아놓은 리스트 
# output: 한 환자의 sessionID들을 주 별로 묶은 것

def divide_sessions(session_arr):
    # 빈 리스트 생성: 각 주별 세션 ID를 저장할 리스트
    session = []
    
    # session_arr에 있는 SessionID를 가진 행들만 필터링하여 새로운 데이터프레임 tf_subset 생성
    # 여기서 VVitalParameter_Systolic, VVitalParameter_Diastolic, VVitalParameter_Pulse 값이 모두 있는 행들만 선택
    tf_subset = total_df.loc[total_df['SessionID'].isin(session_arr)].dropna(
        subset=['VVitalParameter_Systolic', 'VVitalParameter_Diastolic', 'VVitalParameter_Pulse']
    )[['SessionID', 'STime']]
    
    # STime 열을 datetime 형식으로 변환하고, isocalendar() 메서드를 사용하여 주 단위로 그룹화
    # 각 주에 해당하는 데이터프레임들을 리스트로 저장
    dfs = [g for _, g in tf_subset.groupby(pd.to_datetime(tf_subset['STime']).dt.isocalendar().week)]
    
    # 각 주별 데이터프레임을 순회
    for i in range(len(dfs)):
        # 각 주에 속한 SessionID들을 집합으로 만든 후 정렬된 리스트로 변환
        weekly_session = sorted(list(set(dfs[i]['SessionID'])))
        # 주별 세션 리스트를 최종 session 리스트에 추가
        session.append(weekly_session)

    # 주별 세션 리스트를 반환
    return session


In [12]:
### 앞에서 산소포화도 시간을 시:분:초 의 형식으로 바꿨기 때문에 
### : 기준으로 나누어서 레포트에 몇시간 몇분 몇초동안 산소포화도가 95퍼센트 미만이었는지 나타내기 위한 함수 

def sec_to_time(total_sec):
    tmp_time = total_sec.split(':')
    if (tmp_time[0] == '00'):
        if (tmp_time[1] == '00'):
            time_string = tmp_time[2]+'초'
        else:
            time_string = tmp_time[1]+'분 '+tmp_time[2]+'초'
    else:
        time_string = tmp_time[0]+'시간 '+tmp_time[1]+'분 '+tmp_time[2]+'초'

    return time_string

In [13]:
## input
# w_idx(week_index): N주차 (N= 0, 1, 2, 3, 4, 5 ...  )
# w_session_length : 한 주차의 session 개수 

# total table : 주차 정보를 기준으로 만든 table 
# 예) 해당 달이 4주차까지면 
# 1 2
# 3 4  2*2 table 생성됨 

### 각 주차 별 건체중, 적정 증가량에 대한 text 추가하는 코드 ###
### 이 때 한 주의 session 수를 기준으로 칸을 나눔 ### 
def add_table_contents(w_idx, w_session_length):
    # 몇 주 있는지 (w_idx)
    table_position = total_table.rows[(w_idx//2)].cells[(w_idx%2)] # // 는 몫, % 는 나머지 
    table_position._element.clear_content()
    p = table_position.add_paragraph().add_run("{}주차".format(w_idx + 1))
    p.font.size = Pt(12)
    p.bold = True
    about_weight = table_position.add_paragraph().add_run("건체중: {}kg  적정 증가량: {}kg ".format(dry_weight, reasonable_UF))
    
    # p = table_position.add_paragraph()
    # r = p.add_run("{}주차   ".format(w_idx + 1))
    # r.bold = True
    # r.font.size = docx.shared.Pt(12)

    # p.add_run('\t')
    # r = p.add_run("건체중: {}kg".format(dry_weight))
    # p.add_run('\t')
    # r = p.add_run("적정 증가량: {}kg".format(reasonable_UF))
    table = table_position.add_table(rows=1, cols=w_session_length)  

    return table

#### 권고사항 및 추가 권고 사항 관련 함수

In [14]:
# 전체에 대한건 [0, 0, 0, 0]으로 시작하고 하나라도 해당하면 0 -> 1로 바꾸는 걸로 
# 주차별 권고사항 -> 달 별 권고사항으로 수정 

#### month_rec : 환자 상태에 따른 주차별 권고사항 문구를 담기위한 list 
### extra_rec_index : 환자 상태에 따른 추가 문구를 위해 표시하는 index (적절하게 관리되면 0, 개선해야하면 1)
# index 0 = 투석 간 체중 증가
# index 1 = 투석 전 혈압과 투석 중 평균 혈압
# index 2 = 투석 중 저혈압
# index 3 = 투석 중 산소포화도 저하

def print_rcm(month_rec, extra_rec_index, weight_gain, before_dialysis, hypotension, total_spo2, monthly_spo2):

    # index 0 => 투석 간 체중 증가
    if (weight_gain > 0):
        month_rec.append("투석 간 체중 증가량이 적정 증가량 보다 {}회 많습니다.".format(weight_gain))
        extra_rec_index[0] = 1
    else:
        month_rec.append("투석 간 체중 증가가 적절하게 관리되었습니다.")

    # index 1 = 투석 전 혈압과 투석 중 평균 혈압
    if (before_dialysis > 0):
        month_rec.append("투석 전 혈압이 적정 혈압보다 {}회 높았습니다.".format(before_dialysis))
        extra_rec_index[1] = 1
    else:
        month_rec.append("투석 전 혈압이 적당합니다")

    # index 2 = 투석 중 저혈압
    if(len(hypotension) > 0):
        month_rec.append("투석 중 저혈압이 한 달간 {}회 발생했습니다. 가장 많이 발생한 경우는 {}회입니다.".format(sum(hypotension), np.max(hypotension)))
        extra_rec_index[2] = 1
    else:
        month_rec.append("투석 중 저혈압 발생이 없습니다.")
    
    if(total_spo2 > 0): # monthly_spo2는 string이라서 percent로 먼저 판단하고 0 이상이면 저하된 시간 출력하도록 
        month_rec.append("투석 중 산소포화도가 95% 미만으로 저하된 시간이 총 {}입니다.".format(monthly_spo2))
        extra_rec_index[3] = 1
    else:
        month_rec.append("투석 중 산소포화도가 95% 미만으로 저하되지 않았습니다")
    
    return month_rec, extra_rec_index


In [15]:
# 추가 문구 
def print_extra_rcm(extra_rec_index):
    extra_rec = []
    final_rec = ""

    if (extra_rec_index[0] == 1): # index 0 => 투석 간 체중 증가
        final_rec = final_rec + "투석 간 체중 증가 "
        #print(final_rec)
        extra_rec.append("1. 혈압 상승, 투석 중 저혈압, 근육 경련 등의 합병증 발생 위험이 높아지고, 심장 및 뇌혈관 질환의 위험도가 1.5-2배이상 증가합니다. 체중 증가의 원인은 수분과다 섭취, 염분과다 섭취등이 가장 큰 원인이므로 식이 조절이 필수적입니다.")
    else:
        extra_rec.append("1. 수분 섭취 및 염분 섭취가 적절하게 이루어졌을 것으로 판단되므로 열심히 관리하신 것 같습니다.")

    if(extra_rec_index[1] == 1): # index 1 = 투석 전 혈압과 투석 중 평균 혈압
        final_rec = final_rec + ", 투석 전 혈압 "
        #print(final_rec)

        extra_rec.append("2. 투석 전 혈압의 적정값이 140mmHg 이상으로 지속되는 경우 사망률은 70-80% 증가합니다. 혈압을 조절하면 사망률을 44% 줄일 수 있습니다.")
    else: 
        extra_rec.append("2. 투석 전 혈압은 사망률과 관련이 높습니다. 투석 전 혈압을 140 mmHg 미만으로 유지하면 사망률을 44% 줄일 수 있습니다.")

    if(extra_rec_index[2] == 1): # index 2 = 투석 중 저혈압
        final_rec = final_rec + ", 투석 중 저혈압 "
        #print(final_rec)

        extra_rec.append("3. 투석 중 저혈압은 투석 효율을 떨어뜨리고, 심장기능, 심혈관 및 뇌혈관 질환의 위험도를 1.5-2배 이상 높입니다. 투석 중 저혈압은 과다 체중증가, 기존의 심장질환 및 자율신경계 이상, 빈혈, 동맥경화, 혈압약 등에 의해 발생할 수 있습니다.")
    else:
        extra_rec.append("3. 투석 중 저혈압 없이 적절하고 효율적인 투석이 이루어졌습니다.")


    if(extra_rec_index[3] == 1): # index 3 = 투석 중 산소포화도 저하 
        final_rec = final_rec + ", 투석 중 산소포화도 저하 "
        extra_rec.append("4. 정상 산소포화도는 95-100% 입니다. 이보다 낮아지면 심장혈관 및 뇌혈관에 문제를 유발하게 되어 부정맥, 심근경색, 심부전, 뇌경색 등의 위험이 높아집니다.")
    else:
        extra_rec.append("4. 투석 중 산소포화도는 정상 범위로 유지되었습니다. 세포 손상 등의 위험이 없습니다.")

    
    return extra_rec, final_rec
        

#### 기타 오류를 위한 함수

In [16]:
# 이전 데이터 중 같은 날 다른 시간에 측정된 데이터가 있었음 
# 날짜가 같은 것이 있는 지 확인, 날짜가 같으면 같은 날짜 중 뒤에 데이터를 사용 (최신)

def check_same_date(all_session_divide_by_weeks):
    weekly_date_list = []
    for week_idx in range(len(all_session_divide_by_weeks)):
        for session_idx in range(len(all_session_divide_by_weeks[week_idx])):
            sample_date_df = total_df[total_df['SessionID'] == all_session_divide_by_weeks[week_idx][session_idx]].dropna(subset=['VVitalParameter_Systolic', 'VVitalParameter_Diastolic', 'VVitalParameter_Pulse'])
            print(weekly_date_list)
            weekly_date_list.append(list(sample_date_df['StartTime'])[0].strftime("%Y-%m-%d"))
            print(weekly_date_list)
            if (session_idx > 1 and weekly_date_list[week_idx][session_idx] == weekly_date_list[week_idx][session_idx - 1]):
                all_session_divide_by_weeks.pop([week_idx][session_idx - 1])

    print(all_session_divide_by_weeks)
            

## 최종 실행 코드 

In [40]:
#### 연도와 달에 맞춘 폴더 이름  #### 
### 매달 받는 파일에 따라서 수정하면 됨 ###

folder_name = "2024_06_temp" ########################### 레포트 폴더 이름 수정 #################################
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

report_pid = list(set(report_df['P_ID'])) # 레포트를 작성해야하는 PID
# report_pid = list(map(str, report_pid)) # df

# 혈압 관련 시각화 할 때 사용 
pressure_font = {'family': 'Arial',
      'style': 'italic',
      'color': 'blue',
      'size': 20}

days = ['월요일', '화요일',' 수요일', '목요일', '금요일', '토요일', '일요일']

for idx in range(len(report_pid)): #레포트를 작성해야하는 환자
    
    # for file save, 파일 생성
    document = Document("form.docx") # 기본 포맷 불러오기 
    
    # document._body.clear_content() # 처음 엔터 지워주는 역할 
    #r = p.add_run()

    # 제목 (환자 이름)
    print(report_pid[idx])
    patient_name = list(set(total_df[total_df['P_ID'] == report_pid[idx]]['P_Name']))[0]
    print(patient_name)
    #r.add_text("이름: {}".format(patient_name))

    # 제목 (투석 기간)
    dialysis_range = list(set(total_df[total_df['P_ID'] == report_pid[idx]]['TreatmentProgress_Timestamp']))
    #dialysis_range = list(set(total_df[total_df['P_ID'] == report_pid[idx]]['Progress'])) # 파일 column 하나씩 밀려서 원래는 TreatmentProgress_Timestamp가 맞음
    #print(dialysis_range)
    dialysis_range.sort(key=lambda date:datetime.strftime(date, "%Y-%m-%d")) #timestamp sort 하기 
    #print("range", dialysis_range)

    start_date = dialysis_range[0].strftime("%Y-%m-%d")
    end_date = dialysis_range[len(dialysis_range) - 1].strftime("%Y-%m-%d")

    # 데이터 기반으로 파일에 직접 입력하는 코드 
    p = document.add_paragraph("이름: {}    기간: {} ~ {}".format(patient_name, start_date, end_date))
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER

    #r.add_tab()
    #r.add_text("기간: {} ~ {}".format(start_date, end_date))
    
    # 해당 patientID에 해당하는 sessionID 모으기 
    patient_sessions = list(set(total_df[total_df['P_ID'] == report_pid[idx]]['SessionID'])) 
    patient_sessions = sorted(patient_sessions) # 한 환자당 여러 번의 세션이 있음, 시간 순서대로 정렬 
    #print(patient_sessions)

    sessions_by_weeks = divide_sessions(patient_sessions) # 한 주에 해당하는 session 묶기 
    #sessions_by_weeks[1].pop(0) # for (28, 29)
    #print(len(sessions_by_weeks), sessions_by_weeks)
    
    # 날짜가 같은 것이 있는 지 확인, 날짜가 같으면 같은 날짜 중 뒤에 데이터를 사용 (최신)
    #tmp_date_list = check_same_date(sessions_by_weeks)
    #print(tmp_date_list)

    # 주차 별 권고사항 
    rcm_weight_gain = 0; # Preweight - TargetWeight, 건체중 * 0.04 (적정 체중 증가량) / 따라서 (preweight - targetweight)과 reasonable UF 값 비교
    rcm_before_dialysis = 0; # 1주 동안 최초값이 140보다 높은 일 수 
    rcm_hypotension = []; # 1주 동안의 저혈압 횟수
    
    monthly_recommendation = []
    extra_recommendation_index = [0, 0, 0, 0]
    # index 0 = 투석 간 체중 증가
    # index 1 = 투석 전 혈압과 투석 중 평균 혈압
    # index 2 = 투석 중 저혈압
    # index 3 = 투석 중 산소포화도 저하


    # 5주차가 있는 달을 대비해서 아예 변수로 빼기, 보통은 길이 4임 
    if len(sessions_by_weeks) == 5:
        rows_num = 3
    
    elif len(sessions_by_weeks) == 6:
        rows_num = 4
        
    else:
        rows_num = 2
    
    total_table = document.add_table(rows=rows_num, cols=2) # 10월에는 5주차가 있어 아래 한칸을 더 만들어야 함 (6주차도 생겼다... )
    total_table.style = 'Table Grid'

    # 한 환자의 한 주 세션에 대해서 
    #for w_idx in range(1): # 첫째 주만
    for w_idx in range(len(sessions_by_weeks)): # w_idx = week index
        
        first_session = sessions_by_weeks[w_idx][0]
        #print(first_session)
        fs_df = total_df[total_df['SessionID'] == first_session]
        #print(list(fs_df['TargetWeight']))
        dry_weight = list(fs_df['TargetWeight'])[0] # 똑같은 값이라서 , 건체중
        reasonable_UF = round(dry_weight * 0.04 , 2) # 적정 UF, 적정 체중 증가량  
        
        #print("w_idx", w_idx)
        table = add_table_contents(w_idx, len(sessions_by_weeks[w_idx]))

        #for s_idx in range(1): # 한 묶음에 3개의 session이 있다는 전제 
        for s_idx in range(0, len(sessions_by_weeks[w_idx])): # nan값이 있는 것은 drop, 한 환자의 하나의 세션에 대해서 s_idx = session index
            
            sample_df = total_df[total_df['SessionID'] == sessions_by_weeks[w_idx][s_idx]].dropna(subset=['VVitalParameter_Systolic', 'VVitalParameter_Diastolic', 'VVitalParameter_Pulse'])
            ###
            #print(sample_df)

            # 그래프 저장
            pressure_graph = BytesIO()
            uf_graph = BytesIO()
            
            ### 1. systolic, diastolic, pulse 그래프 ###
            week = w_idx + 1 # w_idx가 0부터 시작하기 때문에 
            session_date = list(sample_df['StartTime'])[0].strftime("%Y-%m-%d")
            #print(session_date)

            tmp_session_date = datetime.strptime(session_date,"%Y-%m-%d")
            session_day = days[tmp_session_date.weekday()]

            time = list(sample_df['STime']) # x축

            systolic = list(sample_df['VVitalParameter_Systolic'])
            diastolic = list(sample_df['VVitalParameter_Diastolic'])
            pulse = list(sample_df['VVitalParameter_Pulse'])
            before_dialysis_pressure = systolic[0]

            fig, ax = plt.subplots(figsize=(10, 6))
            plt.ylim([40, 200]) # y축 범위 고정 (논의된 사항)

            plt.plot(systolic, color='blue', marker='o')
            plt.plot(diastolic, color='red', marker='o')
            plt.plot(pulse, color='green', marker='o')

            # 정상 범위 색칠 
            ax.axhspan(90, 140, facecolor='green', alpha=0.1)

            plt.text(0, 180, before_dialysis_pressure, fontdict=pressure_font)
            plt.title(str(session_date) + ' ' + str(session_day), fontdict={'fontsize' : 25})
            # plt.xlabel('time')
            # plt.ylabel('value')
            
            plt.grid(True)
            plt.rcParams['font.size'] = 10
            #plt.legend()
            plt.tight_layout()  # 24.07.10 추가
            plt.savefig(pressure_graph, pad_inches = 0, bbox_inches = 'tight')
            plt.close() # cell을 돌렸을 때 아래에 그림이 안나오게  하기 위해서 
            #plt.show()

            # table.rows[0].cells[s_idx]._element.clear_content() # 쓸데없는 엔터 제거 
            # table.rows[0].cells[s_idx].add_paragraph().add_run().add_picture(pressure_graph, width=Inches(2))
            p = table.rows[0].cells[s_idx].add_paragraph()
            r = p.add_run()
            r.add_picture(pressure_graph, width=Inches(1.6))

            
            ### 2. UF 그래프 (UF_VOLUME 사용) ###
            # 라인 그래프 그리고 아래 채운 후 max 값을 (ml -> kg으로 바꿔서 )

            uf_volume = list(sample_df['UF_Volume'])
            #print("uf_volume", uf_volume)
            total_uf = round(np.max(uf_volume) / 1000, 1)
            #print(total_uf)
            # 투석 간 체중 증가량 
            dialysis_weight_gain = round(list(sample_df['PreWeight'])[0] - list(sample_df['TargetWeight'])[0], 2)


            while 0 in uf_volume:
                uf_volume.remove(0)
            uf_x = range(0, len(uf_volume))

            plt.figure(figsize=(10, 4)) ####### plt.figure(figsize=(10, 4))
            plt.rcParams['font.family'] = 'Malgun Gothic' ### 코드 돌릴 때 폰트는 바꿔야 할듯 !!!!! ###
            plt.rcParams['axes.unicode_minus'] = False
            plt.xticks([]) # x축 글씨 없애기 위해서 
            plt.fill_between(uf_x, uf_volume, color='mediumturquoise')

            try: 
                #plt.text(0, np.min(uf_volume) + 50, "체중 증가량:" + str(dialysis_weight_gain) + "kg", size = 13)
                plt.text(0, np.max(uf_volume) / 2 , "체중 증가량:" + str(dialysis_weight_gain) + "kg", size=25)
                plt.text(len(uf_volume) / 2, np.max(uf_volume) / 2, str(total_uf) + "kg", size=25)
            except ValueError:
                pass
            
            plt.savefig(uf_graph, pad_inches = 0, bbox_inches = 'tight')
            plt.close()
            #plt.show()

            p = table.rows[0].cells[s_idx].add_paragraph()
            r = p.add_run()
            r.add_picture(uf_graph, width=Inches(1.8))
            #print(sample_df)
            session_date_spo2 = list(sample_df['StartTime'])[0].strftime("%Y%m%d")[2:]
            session_date_list = pid_spo2_dict[report_pid[idx]].keys()
            #print("spo2", session_date_spo2)
            #print("date", session_date_list)
            if (session_date_spo2 in session_date_list):
                spo2_seconds = pid_spo2_dict[report_pid[idx]][session_date_spo2]['spo2_under_95_sec']
                spo2_time = sec_to_time(spo2_seconds)
                p = table.rows[0].cells[s_idx].add_paragraph()
                spo2_sentence = p.add_run("산소포화도(95%미만): {}".format(spo2_time))
                spo2_sentence.font.size = Pt(7)
            else:
                #print("passed")
                pass
            

            ######################## for recommendation #########################

            mean_pressure = round(sum(systolic) / len(systolic), 1)

            # 해당 session 내 90 이하로 저하되거나, 직전 혈압에 비해 20이상 감소된 case 의 수 
            hypotension_num = 0
            
            for i in range(len(systolic)):
                #print(systolic[i])
                if (systolic[i] <= 90):
                    hypotension_num += 1        
                elif ((systolic[i - 1] - systolic[i]) >= 20):
                    hypotension_num += 1 
            
            # 투석 중 저혈압 
            rcm_hypotension.append(hypotension_num)

            #rcm
            # 투석 간 체중 증가 
            if(dialysis_weight_gain > reasonable_UF):
                rcm_weight_gain += 1
            
            # 투석 전 혈압이 140보다 높은 경우가 있을 때 
            if (before_dialysis_pressure > 140):
                rcm_before_dialysis += 1
            
            total_spo2_percent = pid_spo2_dict[report_pid[idx]]['whole_spo2_under_95_percent']
            rcm_spo2 = pid_spo2_dict[report_pid[idx]]['whole_spo2_under_95_sec']
            rcm_spo2_time = sec_to_time(rcm_spo2)

            pressure_graph.close()
            uf_graph.close()
            document.save('./{}/report_{}_{}.docx'.format(folder_name, patient_name, report_pid[idx]))     

    # print("rcms", rcm_weight_gain, rcm_before_dialysis, rcm_hypotension)
    rec, extra_recommendation_index = print_rcm(monthly_recommendation, extra_recommendation_index, rcm_weight_gain, rcm_before_dialysis, rcm_hypotension, total_spo2_percent, rcm_spo2_time)
    #print(rec)
    #print("additional_recommendation", extra_recommendation_index)
    
    # 5주차가 있는 달을 대비 
    # 최대한 페이지 수를 줄이기 위해서 만약 5주차까지 있는 경우 
    # 5주차 옆 칸에 권고사항을 적기 위한 코드 (ppt )
    if rows_num == 3:
        table_position = total_table.rows[(5//2)].cells[(5%2)]
        table_position._element.clear_content()
        p = table_position.add_paragraph()
        p.add_run('\n')
        rec_title = p.add_run("권고사항\n")
        rec_title.font.size = Pt(15)
        rec_title.bold = True
        p.add_run('\n')

        for r_idx in range(len(rec)):
            if (r_idx == 0):
                rec_title = p.add_run("1. 투석 간 체중 증가")
                rec_title.bold = True
                p.add_run('\n')
                rec_sentence = p.add_run(rec[r_idx])
                p.add_run('\n')
            
            if (r_idx == 1):
                rec_title = p.add_run("2. 투석 전 혈압과 투석 중 평균 혈압")
                rec_title.bold = True
                p.add_run('\n')
                rec_sentence = p.add_run(rec[r_idx])
                p.add_run('\n')
            
            if (r_idx == 2):
                rec_title = p.add_run("3. 투석 중 저혈압")
                rec_title.bold = True
                p.add_run('\n')
                rec_sentence = p.add_run(rec[r_idx])
                rec_sentence.font.size = Pt(9.5)
                p.add_run('\n')
            
            if (r_idx == 3):
                rec_title = p.add_run("4. 투석 중 산소포화도 저하")
                rec_title.bold = True
                p.add_run('\n')
                # rec_sentence = p.add_run(rec[r_idx])
                rec_sentence = p.add_run(rec[r_idx])
                rec_sentence.font.size = Pt(9.5)
                p.add_run('\n')
            
    else: 
        p = document.add_paragraph()
        p.add_run('\n')
        rec_title = p.add_run("권고사항\n")
        rec_title.font.size = Pt(15)
        rec_title.bold = True
        p.add_run('\n')

        for r_idx in range(len(rec)):
            if (r_idx == 0):
                rec_title = p.add_run("1. 투석 간 체중 증가")
                rec_title.bold = True
                p.add_run('\n')
                rec_sentence = p.add_run(rec[r_idx])
                p.add_run('\n')
            
            if (r_idx == 1):
                rec_title = p.add_run("2. 투석 전 혈압과 투석 중 평균 혈압")
                rec_title.bold = True
                p.add_run('\n')
                rec_sentence = p.add_run(rec[r_idx])
                p.add_run('\n')
            
            if (r_idx == 2):
                rec_title = p.add_run("3. 투석 중 저혈압")
                rec_title.bold = True
                p.add_run('\n')
                rec_sentence = p.add_run(rec[r_idx])
                p.add_run('\n')
            
            if (r_idx == 3):
                rec_title = p.add_run("4. 투석 중 산소포화도 저하")
                rec_title.bold = True
                p.add_run('\n')
                # rec_sentence = p.add_run(rec[r_idx])
                rec_sentence = p.add_run(rec[r_idx])
                p.add_run('\n')
            
    # 추가문구 
    extra_rcm, final_rcm = print_extra_rcm(extra_recommendation_index)
    #print(extra_rcm)

    p = document.add_paragraph()

    for e_idx in range(len(extra_rcm)):
        extra_sentence = p.add_run(extra_rcm[e_idx])
        extra_sentence.font.size = Pt(12)
        p.add_run('\n')
    p.add_run('\n')
    p.add_run("지난 한 달간의 투석 치료에서 ")
    final_sentence = p.add_run(final_rcm)
    final_sentence.font.size = Pt(12)
    final_sentence.bold = True
    final_sentence.font.color.rgb = RGBColor(0x42, 0x24, 0xE9)
    p.add_run("항목에 대한 주의가 필요하겠습니다.")
    document.save('./{}/report_{}_{}.docx'.format(folder_name, patient_name, report_pid[idx])) 
    
    print("{} report complete!!".format(report_pid[idx]))    


14648836
박순식
14648836 report complete!!
21792647
유여희
21792647 report complete!!
28277512
김연희
28277512 report complete!!
28134538
권혁록
28134538 report complete!!
36946314
한유경
36946314 report complete!!
28292751
이선우
28292751 report complete!!
12792977
변민규
12792977 report complete!!
29015700
우민
29015700 report complete!!
10680597
김영숙
10680597 report complete!!
22064408
양주영
22064408 report complete!!
25143836
최정환
25143836 report complete!!
19054621
민기창
19054621 report complete!!
23777567
이만희
23777567 report complete!!
33014946
김영일
33014946 report complete!!
22674470
정달조
22674470 report complete!!
12445992
최숙희
12445992 report complete!!
12883114
김기옥
12883114 report complete!!
11545642
박윤규
11545642 report complete!!
11657901
오현주
11657901 report complete!!
30329904
김상현
30329904 report complete!!
10540978
이경자
10540978 report complete!!
12281011
김영애
12281011 report complete!!
16743737
강형석
16743737 report complete!!
14566972
이영자
14566972 report complete!!
13172413
최충구
13172413 report complete!!
1

## 데이터 없는 환자 확인용 코드

In [1]:
### for missing data check (UF Volume 없는 경우)

date_info = '2024-06-18'
pid_info = 11657901

print("UF_VOLUME_O? : ", sum(total_df[(total_df['STime'].dt.strftime('%Y-%m-%d').str.startswith(date_info)) & (total_df['P_ID'] == pid_info)]['UF_Volume']))
total_df[(total_df['STime'].dt.strftime('%Y-%m-%d').str.startswith(date_info)) & (total_df['P_ID'] == pid_info)]

NameError: name 'total_df' is not defined

In [31]:
### for missing data check ### 

pid_info = 14648836

tmp_df=total_df[(total_df['P_ID'] == pid_info)]
tmp_df['STime'] = pd.to_datetime(tmp_df['STime'])

# 연, 월, 일을 별도의 컬럼으로 추출
tmp_df['ymd'] = tmp_df['STime'].dt.year.astype(str) + '-' + tmp_df['STime'].dt.month.astype(str).str.zfill(2) + '-' + tmp_df['STime'].dt.day.astype(str).str.zfill(2)
set(tmp_df['ymd'])

C:\Users\BMILab\AppData\Local\Temp\ipykernel_23316\4243033933.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['STime'] = pd.to_datetime(tmp_df['STime'])
C:\Users\BMILab\AppData\Local\Temp\ipykernel_23316\4243033933.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['ymd'] = tmp_df['STime'].dt.year.astype(str) + '-' + tmp_df['STime'].dt.month.astype(str).str.zfill(2) + '-' + tmp_df['STime'].dt.day.astype(str).str.zfill(2)


{'2024-06-03',
 '2024-06-05',
 '2024-06-07',
 '2024-06-10',
 '2024-06-12',
 '2024-06-14',
 '2024-06-17',
 '2024-06-19',
 '2024-06-21',
 '2024-06-24',
 '2024-06-26',
 '2024-06-28'}